In [58]:
import mne
import os
from mne.time_frequency import psd_array_multitaper
import numpy as np
import pandas as pd
from scipy.stats import entropy, skew, kurtosis
import csv


## Create seizures csv

#### Variables Defention

In [59]:
# data rows of csv file 
rows_chb05 = [['chb05_06.edf',	417 ,	532],
        ['chb05_13.edf',	1086 ,	1196 ],
        ['chb05_16.edf',	2317 ,	2413 ],
        ['chb05_17.edf',	2451 ,	2571 ],
        ['chb05_22.edf',	2348 ,	2465 ]]

# name of csv file 
filename_chb05 = "chb05_seizure.csv"
# set path to the chb03 folder
chb_chb05 = 'CH05'
# set path to the seizure.csv file
seizure_csv_chb05 = 'chb05_seizure.csv'
# set path to the feature.csv file
feature_csv_chb05 = 'chb05_features.csv'


# data rows of csv file 
rows_chb03 = [['chb03_01.edf',	362,	414],
        ['chb03_02.edf',	731,	796],
        ['chb03_03.edf',	432,	501],
        ['chb03_04.edf',	2162,	2214],
        ['chb03_34.edf',	1982,	2029],
        ['chb03_35.edf',	2592,	2656],
        ['chb03_36.edf',	1725,	1778]]
# name of csv file 
filename_chb03 = "chb03_seizure.csv"
# set path to the chb03 folder
chb_chb03 = 'CH03'
# set path to the seizure.csv file
seizure_csv_chb03 = 'chb03_seizure.csv'
# set path to the feature.csv file
feature_csv_chb03 = 'chb03_features.csv'


##### Choose Variables 

In [60]:
rows = rows_chb03
filename=filename_chb03
chb_folder =chb_chb03 
seizure_csv = seizure_csv_chb03
feature_csv = feature_csv_chb03

In [61]:
# field names 
fields = ['File Name', 'Start', 'End'] 

# # data rows of csv file 



# # name of csv file 
# filename = "chb03_seizure.csv"

# # set path to the chb03 folder
# chb_folder = 'chb03'

# # set path to the seizure.csv file
# seizure_csv = 'chb03_seizure.csv'

# # set path to the feature.csv file
# feature_csv = 'chb03_features.csv'

# writing to csv file 
with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(fields) 

        # writing the data rows 
        csvwriter.writerows(rows)

## Features Extraction

### Create Header for CSV Features 

In [62]:
list_channels=['ch1','ch2','ch3','ch4','ch5','ch6','ch7','ch8','ch9','ch10','ch11',
               'ch12','ch13','ch14','ch15','ch16','ch17','ch18','ch19','ch20','ch21','ch22','ch23']
features=['delta_power','delta_entropy','delta_skewness',
            'theta_power','theta_entropy','theta_skewness',
            'alpha_power','alpha_entropy','alpha_skewness',
            'beta_power','beta_entropy','beta_skewness',
            'mean','std','kurtosis','ptp','var','max','min','argmax','argmin','rms']

new_header=['id','start','end']
for item in list_channels:
    for item2 in  features:
        new_header.append(item+'_'+item2)
new_header.append('target')
print(len(new_header))
with open(feature_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(new_header)

510


In [63]:
freq_bands = {'delta': [0.5, 4], 'theta': [4, 7], 'alpha': [7, 15], 'beta': [15, 30]}

In [64]:
def extract_spectral_features(raw, freq_bands):
    spectral_features = []
    raw.filter(1, 120)
    raw.set_eeg_reference('average')
    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
    psds, freqs = psd_array_multitaper(raw.get_data()[eeg_picks], sfreq=raw.info['sfreq'], fmin=0.5, fmax=30) # type: ignore

    for band, freq_range in freq_bands.items():
        band_data = raw.copy().filter(freq_range[0], freq_range[1], picks=eeg_picks).get_data()

        if np.isnan(band_data).any():
            print(f"No data available for {band} band.")
            continue

        band_power = np.mean(np.abs(band_data) ** 2, axis=1)
        band_entropy = entropy(np.abs(band_data) ** 2, axis=1)
        band_skewness = skew(band_data, axis=1)

        # spectral_features += ["freq",band_power, band_entropy, band_skewness]
        # spectral_features.append(band_power[0][ch],)
        for ch in range(23):
            # print(band_power.shape)
            spectral_features.append(band_power[ch])
            spectral_features.append(band_entropy[ch])
            spectral_features.append(band_skewness[ch])
            
    return spectral_features

In [65]:
def extract_time_features(raw):
    time_features = []

    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
    eeg_data = raw.get_data()[eeg_picks]

    for channel_data in eeg_data:
        time_features.append(np.mean(channel_data)) 

        time_features.append(np.std(channel_data))

        time_features.append(kurtosis(channel_data))
            
        time_features.append(np.ptp(channel_data,axis=-1))

        time_features.append(np.var(channel_data,axis=-1))

        time_features.append(np.min(channel_data,axis=-1))

        time_features.append(np.max(channel_data,axis=-1))

        time_features.append(np.argmin(channel_data,axis=-1))

        time_features.append(np.argmax(channel_data,axis=-1))

        time_features.append(np.sqrt(np.mean(channel_data ** 2,axis=-1)))  #root mean sqaured #type: ignore

    return time_features

In [66]:
def extract_features(patient, file, start=0, end=3599, window=20, seizure_start=[0], seizure_end=[0]):

    
    raw = mne.io.read_raw_edf(f'{patient}/{file}', preload=True)
    temp=start


    while temp<end:
        all_features = [file,temp,temp+window]
        # create a copy of the original object
        raw_copy = raw.copy()

        # apply slicing on the copy
        if (temp+window)<end:
            current_data = raw_copy.crop(tmin=temp, tmax=(temp+window))
        else:
            current_data = raw_copy.crop(tmin=temp, tmax=end)


        spectral_features = extract_spectral_features(current_data, freq_bands)
        time_features = extract_time_features(current_data)
        
        all_features.extend( spectral_features + time_features)

        target=0
        for seiz_start,seiz_end in zip(seizure_start,seizure_end):
            if temp in range(seiz_start,(seiz_end-1)):
                target = 1
        all_features.append(target) 

        temp+=window
        
        data = [float(item) if isinstance(item, np.float64) else item for item in all_features] # type: ignore
        with open(feature_csv, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(data)

In [67]:
# read the seizure.csv file as a pandas dataframe
seizure_df = pd.read_csv(seizure_csv)

# get a list of all the files in the chb03 folder with the '.edf' extension
edf_files_list = [f for f in os.listdir(chb_folder) if f.endswith('.edf')]

# loop through each file in the chb03 folder
for file in edf_files_list:
    # check if the file exists in the seizure.csv file
    if file in seizure_df['File Name'].values:
        # if the file exists, get the corresponding start and end times from the CSV file
        row = seizure_df.loc[seizure_df['File Name'] == file]
        seizure_start = row['Start'].values
        seizure_end = row['End'].values
        extract_features(patient=chb_folder ,file=file,seizure_start=seizure_start,seizure_end=seizure_end)
    else:
        # if the file does not exist, assign both variables a value of zero
        extract_features(patient=chb_folder ,file=file)
        
    
    # do something with the seizure_start and seizure_end variables
    # for example, print them to the console
    print(f'File: {file}, Seizure Start: {seizure_start}, Seizure End: {seizure_end}')

Extracting EDF parameters from c:\Users\hp\Documents\GitHub\CDSS-Final-project\CH03\chb03_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


C:\Users\hp\AppData\Local\Temp\ipykernel_5232\2263733200.py:4: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(f'{patient}/{file}', preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 120.00 Hz
- Upper transition bandwidth: 8.00 Hz (-6 dB cutoff frequency: 124.00 Hz)
- Filter length: 845 samples (3.301 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
    Using multitaper spectrum estimation with 7 DPSS windows
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 15 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 15.00
- Lower transition bandwidth: 3.75 Hz (-6 dB cutoff frequency: 13.12 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 227 samples (0.887 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband 

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 120.00 Hz
- Upper transition bandwidth: 8.00 Hz (-6 dB cutoff frequency: 124.00 Hz)
- Filter length: 845 samples (3.301 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
    Using multitaper spectrum estimation with 7 DPSS windows
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 1691 samples (6.605 s)

Filtering raw data in 1 contiguous segment
Settin

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 120.00 Hz
- Upper transition bandwidth: 8.00 Hz (-6 dB cutoff frequency: 124.00 Hz)
- Filter length: 845 samples (3.301 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
    Using multitaper spectrum estimation with 7 DPSS windows
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with